<a href="https://colab.research.google.com/github/saffa19/shippy/blob/main/ais_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install websockets
#!pip install asyncio
#!pip install nest_asyncio
#!pip install descartes
#!pip install geopandas

In [2]:
import configparser

def fetch_key(key):

    cfg = configparser.ConfigParser()
    cfg.read('config.cfg')

    return cfg.get('KEYS', f'{key}', raw='')

In [3]:
key = fetch_key('ais')

In [4]:
#!rm -rf tracking
#!mkdir tracking
!rm -rf images
!mkdir images

In [5]:
import os
import json
import pandas as pd

def tracker(msg):
    vessel_id = msg['Message']['PositionReport']['UserID']
    path = f"./tracking/{vessel_id}.json"
    position_report = msg['Message']['PositionReport']
    metadata = msg['MetaData']
    entry = position_report | metadata
    #print(msg)

    if not os.path.exists(path):
        #print(f'creating {path}')
        with open(path, 'w') as file:
            json.dump([entry], file, indent = 4)
    elif os.path.exists(path):
        #print(f'updating {path}')
        with open(path, 'r') as file:
            data = json.load(file)
        data.append(entry)
        with open(path, 'w') as file:
            json.dump(data, file, indent = 4)

    else:
        print(f'bad vessel {vessel_id}')

    #df = pd.read_json(path)
    #print(df.head())

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import json
from shapely.geometry import Point, Polygon
import pylab as pl
from IPython import display

def mapper(vessel_id):
    with open(f'/content/tracking/{vessel_id}.json', 'r') as file:
        data = json.load(file)
    df = pd.DataFrame(data)
    geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
    geo_df = gpd.GeoDataFrame(df, geometry=geometry)
    #geo_df.head()

    map = gpd.read_file('/content/maps/ne_10m_ocean.shp')

    fig, ax = plt.subplots(figsize = (7,7))
    map.plot(ax = ax, alpha = 0.4, color ='grey')
    geo_df.plot(geo_df['Sog'], ax = ax, markersize = 10,legend=True, legend_kwds={"label": "Speed over Ground (SoG)", "orientation": "horizontal"})
    plt.title(f'{df["UserID"].iloc[0]} {df["ShipName"].iloc[0].strip()}')
    #plt.title('Ships')
    posy, posx = geo_df['Latitude'].iloc[-1], geo_df['Longitude'].iloc[-1]
    plt.xlim(posx-.8, posx+.8)
    plt.ylim(posy-.8, posy+.8)
    #plt.xlim(110,125)
    #plt.ylim(15,30)
    plt.xlabel("Latitude")
    plt.ylabel("Longitude")

    ax.annotate(geo_df['time_utc'].iloc[0][:16], (geo_df['Longitude'].iloc[0], geo_df['Latitude'].iloc[0]), xytext=(posx+.82, geo_df['Latitude'].iloc[0]), arrowprops = dict(arrowstyle="->"))
    ax.annotate(geo_df['time_utc'].iloc[-1][:16], (posx,posy), xytext=(posx+.82, posy), arrowprops = dict(arrowstyle="->"))#ax.legend()

    plt.savefig(f'/content/images/{df["UserID"].iloc[0]} {df["time_utc"].iloc[0][:10]}.png', bbox_inches='tight')
    #plt.show()
    plt.close()

In [ ]:
import asyncio, nest_asyncio
import websockets
import json
from datetime import datetime, timezone
from IPython.display import clear_output


nest_asyncio.apply()

async def connect_ais_stream():

    async with websockets.connect("wss://stream.aisstream.io/v0/stream") as websocket:
        subscribe_message = {"APIKey": key, "BoundingBoxes": [[[20,118], [24,119]]]}
        subscribe_message_json = json.dumps(subscribe_message)
        await websocket.send(subscribe_message_json)

        async for message_json in websocket:
            message = json.loads(message_json)
            message_type = message["MessageType"]
            #print(message)
            if message_type == "PositionReport":
                # the message parameter contains a key of the message type which contains the message itself
                ais_message = message['Message']['PositionReport']
                tracker(message)
                mapper(ais_message['UserID'])
                #print(f'Tracked vessels: {len([name for name in os.listdir("/content/tracking/") if os.path.isfile(name)])}')
                #print(f"[{datetime.now(timezone.utc)}] ShipId: {ais_message['UserID']} Latitude: {ais_message['Latitude']} Latitude: {ais_message['Longitude']}")
            else:
                clear_output(wait=True)
                print(message)


loop = asyncio.get_event_loop()
loop.run_until_complete(connect_ais_stream())
loop.close()

{'Message': {'StandardClassBPositionReport': {'AssignedMode': False, 'ClassBBand': True, 'ClassBDisplay': False, 'ClassBDsc': True, 'ClassBMsg22': False, 'ClassBUnit': True, 'Cog': 279.1, 'CommunicationState': 393222, 'CommunicationStateIsItdma': True, 'Latitude': 23.77729833333333, 'Longitude': 118.04919666666666, 'MessageID': 18, 'PositionAccuracy': False, 'Raim': False, 'RepeatIndicator': 0, 'Sog': 3.5, 'Spare1': 5, 'Spare2': 0, 'Timestamp': 36, 'TrueHeading': 511, 'UserID': 412445912, 'Valid': True}}, 'MessageType': 'StandardClassBPositionReport', 'MetaData': {'MMSI': 412445912, 'MMSI_String': 412445912, 'ShipName': '', 'latitude': 23.77729833333333, 'longitude': 118.04919666666666, 'time_utc': '2023-11-29 22:24:37.446762076 +0000 UTC'}}
